<img src="../squeemos_blk.png" width=200 height=200 />

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # makes the notebook fill the whole window

import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import seaborn as sns; sns.set()
import csv

from PIL import Image, ImageDraw
import os
import shutil
import glob

from mpl_toolkits.basemap import Basemap

from BetterMap import CreateMapBackground

import math

import functools
import operator

import datetime

from pathlib import Path

from Utils import *

#there is currently a bug in panda's scatter_matrix that produces a warning that has no affect on what I'm doin :)
import warnings
warnings.filterwarnings("ignore")

## Clustering functions

In [ ]:
# Return kmeans cluster centers of the given dataframe
def cluster_df(df):
    if df.shape[0] > 0:
        num_clusters = 5 #TODO: use dbscan for optimal cluster number
        if df.shape[0] < num_clusters:
            num_clusters = df.shape[0]
        kmeans = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=10, algorithm='full')
        kmeans.fit(df[['Lat', 'Long']])
        df['cluster_label'] = kmeans.fit_predict(df[['Lat', 'Long']])
        centers = kmeans.cluster_centers_
        return centers
    result = np.empty((2,1))
    result[:] = np.NaN
    return result

#These 3 functions finds clustered centers of specific parts of the storm
def create_inner_df(df):
    centers = []
    df_inner_temp = df.copy()
    df_inner_temp = df_inner_temp[df_inner_temp['distance'] <= 100]

    df_inner = df.copy()
    df_inner = df_inner[(df_inner['distance'] <= 100) & (df_inner['segment'] == current_segment)]
    centers.append(cluster_df(df_inner))
    
    #flatten the list
    centers = functools.reduce(operator.iconcat, centers, [])
    #drop nan
    centers = [x for x in centers if str(x) != 'nan']
    return centers

def create_rainband_df(df):
    centers = []
    df_rainband_temp = df.copy()
    df_rainband_temp = df_rainband_temp[(df_rainband_temp['distance'] >= 200) & (df_rainband_temp['distance'] <= 400)]

    df_rainband = df.copy()
    df_rainband = df_rainband[(df_rainband['distance'] >= 200) & (df_rainband['distance'] <= 400)]
    centers.append(cluster_df(df_rainband))
    
    #flatten the list
    centers = functools.reduce(operator.iconcat, centers, [])
    #drop nan
    centers = [x for x in centers if str(x) != 'nan']
    return centers

def create_outer_df(df):
    centers = []
    df_outer_temp = df.copy()
    df_outer_temp = df_outer_temp[(df_outer_temp['distance'] >= 400) & (df_outer_temp['distance'] <= 600)]

    df_outer = df.copy()
    df_outer = df_outer[(df_outer['distance'] >= 400) & (df_outer['distance'] <= 600)]
    centers.append(cluster_df(df_outer))
    
    #flatten the list
    centers = functools.reduce(operator.iconcat, centers, [])
    #drop nan
    centers = [x for x in centers if str(x) != 'nan']
    return centers

def segment_df(df, minutes_between):
    cluster = 0
    df_time = pd.DataFrame()
    df_time['segment'] = 0
    for x in range(len(df['Month'].unique())):
        month = df['Month'].unique()[x]
        for y in range(len(df[df['Month'] == df['Month'].unique()[x]]['Day'].unique())):
            day = df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]
            for z in range(len(df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique())):
                hour = df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique()[z]
                for w in range(0,60,minutes_between):
                    minute = w
                    temp = df[(df['Month'] == month) & (df['Day'] == day) & (df['Hour'] == hour) & (df['Min'] >= minute) & (df['Min'] < minute + minutes_between)]
                    temp['segment'] = cluster
                    df_time = df_time.append(temp, ignore_index=True)
                    cluster+=1
    return df_time

## Data loading and verification

In [ ]:
#load in the data
df = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_WWLLN_Locations.txt',header=None,names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"],low_memory=False, delim_whitespace=True)
df.dropna(axis=0,how='any',subset=['Lat','Long'],inplace=True)

df_center = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_Reduced_Trackfile.txt',header=None,names=["Year","Month","Day","Hour","Lat","Long","Min_Pressure","Max_Winds","Unused"],low_memory=False,sep='\t')
df_center.dropna(axis=0,how='any',subset=['Lat','Long'],inplace=True)

In [ ]:
display(df.info())
display(df_center.info())

## Proof of concept/test

In [ ]:
def test():
    #make a dataframe that denotes clusters based on a time interval - a new cluster every x amount of minutes
    minutes_between = 30

    df_time = segment_df(df, minutes_between)
    df_time['distance'] = np.sqrt((df_time['Dist_East_West'] * df_time['Dist_East_West']) + 
                                    (df_time['Dist_North_South'] * df_time['Dist_North_South']))
    df_time.info()

    #apply date restriction
    year, month, day, hour, minute = 2017, 9, 7, 16, 30

    temp = df_time[(df_time['Year'] == year) &
                       (df_time['Month'] == month) &
                       (df_time['Day'] == day) &
                       (df_time['Hour'] == hour) &
                       (df_time['Min'] == minute)]
    temp.info()

    inner_clusters = create_inner_df(temp)
    rainband_clusters = create_rainband_df(temp)
    outer_clusters = create_outer_df(temp)

    inner_centers = pd.DataFrame(data=inner_clusters, columns=["Lat", "Long"])
    rainband_centers = pd.DataFrame(data=rainband_clusters, columns=["Lat", "Long"])
    outer_centers = pd.DataFrame(data=outer_clusters, columns=["Lat", "Long"])

    ln = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_WWLLN_Locations.txt',header=None,names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"],low_memory=False,sep=' ')
    minLong_, minLat_, maxLong_, maxLat_ = ln['Long'].min(), ln['Lat'].min(), ln['Long'].max(), ln['Lat'].max()
    buffer = 10

    plt.figure(figsize=(20,20))
    m = Basemap(llcrnrlon=minLong_-buffer, llcrnrlat=minLat_-buffer,urcrnrlon=maxLong_+buffer,urcrnrlat=maxLat_+buffer,lon_0=0,lat_0=0)
    m.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
    m.fillcontinents(color='grey', alpha=0.7, lake_color='grey')
    m.drawcoastlines(linewidth=0.1, color="white")

    # Plot the lightning data
    m.plot(temp['Long'], temp['Lat'], linestyle='none', marker="o", markersize=15, alpha=0.3, c="yellow", markeredgecolor="black", markeredgewidth=1)
    m.plot(inner_centers['Long'], inner_centers['Lat'], linestyle='none', marker=".", markersize=8, alpha=0.3, c="red", markeredgecolor="black", markeredgewidth=1)
    m.plot(rainband_centers['Long'], rainband_centers['Lat'], linestyle='none', marker=".", markersize=8, alpha=0.3, c="green", markeredgecolor="black", markeredgewidth=1)
    m.plot(outer_centers['Long'], outer_centers['Lat'], linestyle='none', marker=".", markersize=8, alpha=0.3, c="blue", markeredgecolor="black", markeredgewidth=1)

    plt.show()

In [ ]:
#test()

## Clustering to create a gif

In [ ]:
minutes_between = 30

ln = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_WWLLN_Locations.txt',header=None,names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"],low_memory=False,sep=' ')
minLong_, minLat_, maxLong_, maxLat_ = ln['Long'].min(), ln['Lat'].min(), ln['Long'].max(), ln['Lat'].max()
buffer = 10

df_time = segment_df(df, minutes_between)
df_time['distance'] = np.sqrt((df_time['Dist_East_West'] * df_time['Dist_East_West']) + 
                                (df_time['Dist_North_South'] * df_time['Dist_North_South']))
display(df_time.info())
df_time.head(5)

In [ ]:
%%time

year = 2017
edges = (minLong_,maxLong_,minLat_,maxLat_)
di = './data/kmeans/final_gif/'

end = df_time['segment'].max()
if math.isnan(end):
    end = 0
for current_segment in range(1, int(end)):
    temp = df_time[(df_time['segment'] == current_segment)]
    if(temp.shape[0] > 0):
        inner_clusters = create_inner_df(temp)
        inner_clusters = [x for x in inner_clusters if str(x[0]) != 'nan']
        rainband_clusters = create_rainband_df(temp)
        rainband_clusters = [x for x in rainband_clusters if str(x[0]) != 'nan']
        outer_clusters = create_outer_df(temp)
        outer_clusters = [x for x in outer_clusters if str(x[0]) != 'nan']

        inner_centers = pd.DataFrame(data=inner_clusters, columns=["Lat", "Long"])
        rainband_centers = pd.DataFrame(data=rainband_clusters, columns=["Lat", "Long"])
        outer_centers = pd.DataFrame(data=outer_clusters, columns=["Lat", "Long"])

        fig = plt.figure(figsize=(30,15))
        m = CreateMapBackground(edges,buffer)

        # Plot the lightning data
        m.plot(temp['Long'], temp['Lat'], linestyle='none', marker="o", markersize=15, alpha=0.3, c="yellow", markeredgecolor="black", markeredgewidth=1)
        m.plot(inner_centers['Long'], inner_centers['Lat'], linestyle='none', marker=".", markersize=8, alpha=0.3, c="red", markeredgecolor="black", markeredgewidth=1)
        m.plot(rainband_centers['Long'], rainband_centers['Lat'], linestyle='none', marker=".", markersize=8, alpha=0.3, c="green", markeredgecolor="black", markeredgewidth=1)
        m.plot(outer_centers['Long'], outer_centers['Lat'], linestyle='none', marker=".", markersize=8, alpha=0.3, c="blue", markeredgecolor="black", markeredgewidth=1)
        
        month, day, hour, minute = int(temp['Month'].iloc[0]), int(temp['Day'].iloc[0]), int(temp['Hour'].iloc[0]), int(temp['Min'].iloc[0])
        #Save and close the figure
        plt.savefig('{}{}_{}_{}_{}_{}'.format(di,year,month,day,hour,minute),bbox_inches='tight')
        plt.close(fig)

In [ ]:
MakeGif('./data/kmeans/final_gif', './data/kmeans/', 'kmeans', 100, 'png')

In [ ]:
ClearDir('./data/kmeans/final_gif')

In [ ]:
#DEPRECATED
#elbow graph for visual estimation of ideal cluster count
# clusters = range(1,16)

# kmeans = [KMeans(n_clusters=i) for i in clusters]

# y = df[['Lat']]
# x = df[['Long']]

# score = [kmeans[i].fit(y).score(y) for i in range(len(kmeans))]

# plt.plot(clusters, score)

# plt.show()

# #kmeans using the set cluster count
# kmeans = KMeans(n_clusters=4, init='k-means++')
# cluster_df = coord_df
# kmeans.fit(cluster_df)
# cluster_df['segment'] = kmeans.fit_predict(cluster_df)
# inertia = kmeans.inertia_
# labels = kmeans.predict(cluster_df[cluster_df.columns[:2]])
# centers = kmeans.cluster_centers_

# cluster_df.plot.scatter(x='Lat',y='Long', c=labels, s=50, cmap='spring', figsize=(20,20))
# plt.scatter(centers[:, 0], centers[:, 1], c='black', s= 200, alpha=.5)

# plt.show()

# for n_clusters in range(1,16):
#     kmeans = KMeans(n_clusters=n_clusters, init='k-means++')
#     kmeans.fit(cluster_df)
#     inertia = kmeans.inertia_
#     print("Cluster count: ", n_clusters, "\tInertia: ", inertia)